# 动态规划
值迭代(value iteration)和策略迭代(policy iteration)都是基于动态规划(Dynamic Programming)的强化学习算法
值迭代和策略迭代都是基于模型的
<br>值迭代基于贝尔曼最优方程，策略迭代基于贝尔曼方程

# 值迭代 Value Iteration

## 悬崖漫步环境

In [1]:
import copy
class CliffWalkingEnv:
    """ 悬崖漫步环境"""
    def __init__(self, ncol=12, nrow=4):
        self.ncol = ncol  # 定义网格世界的列
        self.nrow = nrow  # 定义网格世界的行
        # 转移矩阵P[state][action] = [(p, next_state, reward, done)]包含下一个状态和奖励
        self.P = self.createP()

    def createP(self):
        # 初始化P
        #P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]
        nS = self.nrow * self.ncol   # 48 个状态
        nA = 4                       # 4 个动作
        P = []                       # 最外层大列表
        for i in range(nS):          # i = 0,1,2,…,47
            tmp = []                 # 准备放当前状态的所有动作
            for j in range(nA):      # j = 0,1,2,3
                tmp.append([])       # 先放一个空列表
            P.append(tmp)            # 把这一行挂到大列表上

        # 4种动作, change[0]，即[0,-1]是向上走,change[1]:下, change[2]:左, change[3]:右。坐标系原点(0,0)
        # 定义在左上角，坐标系原点 (0,0) 定义在左上角，往右和下坐标值会变大
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]

        for i in range(self.nrow):
            for j in range(self.ncol):
                for a in range(4):
                    # 位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
                    if i == self.nrow - 1 and j > 0:
                        P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0,True)]
                        continue
                    # 其他位置
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))
                    next_state = next_y * self.ncol + next_x
                    reward = -1
                    done = False
                    # 下一个位置在悬崖或者终点
                    if next_y == self.nrow - 1 and next_x > 0:
                        done = True
                        if next_x != self.ncol - 1:  # 下一个位置在悬崖
                            reward = -100
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]
        return P

In [2]:
env = CliffWalkingEnv()
env.P

[[[(1, 0, -1, False)],
  [(1, 12, -1, False)],
  [(1, 0, -1, False)],
  [(1, 1, -1, False)]],
 [[(1, 1, -1, False)],
  [(1, 13, -1, False)],
  [(1, 0, -1, False)],
  [(1, 2, -1, False)]],
 [[(1, 2, -1, False)],
  [(1, 14, -1, False)],
  [(1, 1, -1, False)],
  [(1, 3, -1, False)]],
 [[(1, 3, -1, False)],
  [(1, 15, -1, False)],
  [(1, 2, -1, False)],
  [(1, 4, -1, False)]],
 [[(1, 4, -1, False)],
  [(1, 16, -1, False)],
  [(1, 3, -1, False)],
  [(1, 5, -1, False)]],
 [[(1, 5, -1, False)],
  [(1, 17, -1, False)],
  [(1, 4, -1, False)],
  [(1, 6, -1, False)]],
 [[(1, 6, -1, False)],
  [(1, 18, -1, False)],
  [(1, 5, -1, False)],
  [(1, 7, -1, False)]],
 [[(1, 7, -1, False)],
  [(1, 19, -1, False)],
  [(1, 6, -1, False)],
  [(1, 8, -1, False)]],
 [[(1, 8, -1, False)],
  [(1, 20, -1, False)],
  [(1, 7, -1, False)],
  [(1, 9, -1, False)]],
 [[(1, 9, -1, False)],
  [(1, 21, -1, False)],
  [(1, 8, -1, False)],
  [(1, 10, -1, False)]],
 [[(1, 10, -1, False)],
  [(1, 22, -1, False)],
  [(1, 9, -

## Value Iteration 基于贝尔曼最优

In [3]:
class ValueIteration:
    """ 价值迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 初始化,所有state的state value都为0，48个
        self.theta = theta  # 价值收敛阈值，当两次迭代的最大差值 < θ 时停止
        self.gamma = gamma
        # 初始化策略，48个None,一维列表,后面会存列表进去，变成二维的
        self.pi = [None for i in range(self.env.ncol * self.env.nrow)]  #长度48的列表，最后放“最优策略”，pi里面有48个元素。

    def value_iteration(self):
        counter = 0  #计数器，用来看迭代了多少轮
        while 1:     #只能靠内部 break 跳出来
            max_diff = 0    #最大差值
            new_v = [0] * self.env.ncol * self.env.nrow
            for s in range(self.env.ncol * self.env.nrow):
                qsa_list = []                           # 开始计算状态s下的所有Q(s,a)价值
                for a in range(4):                      #计算当前状态的4个动作的q,即qsa
                    qsa = 0              #计算当前状态和当前动作的qsa，因为一个动作可能有多条随机转移（虽然本环境只有 1 条），要把期望 Q 值“加权求和”。
                    for p, next_state, r, done in self.env.P[s][a]:   #调取P中的(p,next_state,reward,done)
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))  #☆
                        #(1 - done)，如果这条转移已经让回合结束 (done=1)，就把“未来回报”整项抹成 0；否则保持原样。
                    qsa_list.append(qsa)                             #这一行和下一行代码是价值迭代和策略迭代的主要区别
                new_v[s] = max(qsa_list)                             #重点，这就是贝尔曼最优公式
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))  #把每个状态新旧价值差的绝对值都算出来，只保留最大的那一个作为本轮迭代的变化量 max_diff
            self.v = new_v
            if max_diff < self.theta: break                          # 满足收敛条件,退出评估迭代
            counter += 1
        print("价值迭代一共进行%d轮" % counter)
        self.get_policy()

    def get_policy(self): #不参与训练，根据价值函数导出一个贪婪策略
        for s in range(self.env.nrow * self.env.ncol):
            qsa_list = []
            for a in range(4):
                qsa = 0
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)
            maxq = max(qsa_list)
            count_q = qsa_list.count(maxq)     # 计算有几个动作得到了最大的Q值
            # 让这些动作均分概率
            self.pi[s] = [1 / count_q if q == maxq else 0 for q in qsa_list]

## 算法文字描述
先随便初始化state value和policy(其实不用初始化策略)
<br>然后计算每个state的每个动作的q(s,a),根据贝尔曼公式算，要用到初始化的v。然后把最大的q(s,a)直接赋给v(s)，依据是贝尔曼最优
<br>这样就的得到了每个state的新v，跟上一次的作比较，如果max_diff不小于theta,
<br>再计算每个state的每个动作的q(s,a),根据贝尔曼公式算，要用到刚刚的新v,然后把最大的q(s,a)直接赋给v(s)，又得到新一次的v
<br>然后这一次的新v和上一次的v做比较，如果max_diff小于theta，
<br>然后根据最后一次的v计算每个每个状态每个动作的q(s,a),然后根据最大的q导出策略

In [4]:
def print_agent(agent, action_meaning, disaster=[], end=[]):
    print("状态价值：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 为了输出美观,保持输出6个字符
            print('%6.6s' % ('%.3f' % agent.v[i * agent.env.ncol + j]), end=' ')
        print()

    print("策略：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 一些特殊的状态,例如悬崖漫步中的悬崖
            if (i * agent.env.ncol + j) in disaster:
                print('****', end=' ')
            elif (i * agent.env.ncol + j) in end:  # 目标状态
                print('EEEE', end=' ')
            else:
                a = agent.pi[i * agent.env.ncol + j]
                pi_str = ''
                for k in range(len(action_meaning)):
                    pi_str += action_meaning[k] if a[k] > 0 else 'o'
                print(pi_str, end=' ')
        print()


In [5]:
#实例化环境
env = CliffWalkingEnv()
#动作含义
action_meaning = ['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
#实例化算法
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
#打印每个状态的state value和策略
print_agent(agent, action_meaning, list(range(37, 47)), [47])

价值迭代一共进行14轮
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 
